In [ ]:
!pip install xgboost
!pip install shap

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score, accuracy_score, f1_score, mean_squared_error, plot_confusion_matrix
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
import xgboost as xgb
from scipy.stats import randint, uniform
import shap

# Data Collection and Cleaning

In [ ]:
# Import data
compustat1990 = pd.read_csv('C:\\Users\\andre\\OneDrive\\Documents\\SchoolWork\\2021 Spring Semester\\CIS 519\\Final Project\\compustat1990-2000.csv', low_memory=False)
compustat2000 = pd.read_csv('C:\\Users\\andre\\OneDrive\\Documents\\SchoolWork\\2021 Spring Semester\\CIS 519\\Final Project\\compustat2000-2010.csv', low_memory=False)
compustat2010 = pd.read_csv('C:\\Users\\andre\\OneDrive\\Documents\\SchoolWork\\2021 Spring Semester\\CIS 519\\Final Project\\compustat2010-2021.csv', low_memory=False)

# Import intermediate/joiner data
compustatjoin = pd.read_csv('C:\\Users\\andre\\OneDrive\\Documents\\SchoolWork\\2021 Spring Semester\\CIS 519\\Final Project\\compustatjoin.csv', low_memory=False)
ibesjoin = pd.read_csv('C:\\Users\\andre\\OneDrive\\Documents\\SchoolWork\\2021 Spring Semester\\CIS 519\\Final Project\\ibesjoin.csv', low_memory=False)

# Import forecast data
ibes1990 = pd.read_csv('C:\\Users\\andre\\OneDrive\\Documents\\SchoolWork\\2021 Spring Semester\\CIS 519\\Final Project\\ibes1990-2000.csv', low_memory=False)
ibes2000 = pd.read_csv('C:\\Users\\andre\\OneDrive\\Documents\\SchoolWork\\2021 Spring Semester\\CIS 519\\Final Project\\ibes2000-2010.csv', low_memory=False)
ibes2010 = pd.read_csv('C:\\Users\\andre\\OneDrive\\Documents\\SchoolWork\\2021 Spring Semester\\CIS 519\\Final Project\\ibes2010-2020.csv', low_memory=False)

# Concatenate timeseries data
compustat_df = pd.concat([compustat1990, compustat2000, compustat2010])
ibes_df = pd.concat([ibes1990, ibes2000, ibes2010])

# Merge data and forecasts according to appropriate naming conventions
compustat_df = compustat_df.merge(compustatjoin,on='gvkey',how='inner')
ibes_df = ibes_df.merge(ibesjoin,on='OFTIC',how='inner')

# Filter for only quarterly reports
quarterly_ibes = ibes_df[ibes_df['FISCALP'] == 'QTR']

# Merge forecasts and data
merged_df = compustat_df.merge(quarterly_ibes, left_on=['permno','datadate'], right_on=['PERMNO','FPEDATS'], how='inner')

# Save merged data locally
# compression_opts = dict(method='zip',
#                         archive_name='out.csv')  
# merged_df.to_csv('C:\\Users\\andre\\OneDrive\\Documents\\SchoolWork\\2021 Spring Semester\\CIS 519\\Final Project\\merged_data.zip', index=False, compression=compression_opts)

# Read merged_data.zip (saved above)
# import zipfile
# zf = zipfile.ZipFile('C:\\Users\\andre\\OneDrive\\Documents\\SchoolWork\\2021 Spring Semester\\CIS 519\\Final Project\\merged_data.zip')
# merged_df = pd.read_csv(zf.open('out.csv'))

# Add Ground Truth
merged_df = merged_df.dropna(subset=['atq','MEANEST','ACTUAL'])
merged_df = merged_df[merged_df['atq'] != 0]
merged_df['AFE'] = ((merged_df['ACTUAL'] - merged_df['MEANEST'])*merged_df['cshoq'])

# Allocate memory
del compustat1990, compustat2000, compustat2010
del compustatjoin, ibesjoin
del ibes1990, ibes2000, ibes2010
del compustat_df, quarterly_ibes       

# Clean data
cleaned_df = merged_df.dropna(axis=1, how='all')
cleaned_df = cleaned_df.dropna(subset=['AFE'])
cleaned_df = cleaned_df.drop(columns=ibes_df.columns)
cleaned_df = cleaned_df.drop(columns=['gvkey','indfmt','cusip','tic','datadate','fyearq','fqtr','fyr','conm','cik','exchg','fic','fyrc','datacqtr','datafqtr','rdq'])
for column in cleaned_df.columns:
    if (column != 'atq' and column != 'permno'):
        cleaned_df[column] = cleaned_df[column]/cleaned_df['atq']

# Remove outliers
cleaned_df = cleaned_df[np.abs(cleaned_df['AFE']) < 10000]

# Regression on AFE

In [ ]:
# Split data into training and testing (grouped by company)
permno_df = cleaned_df['permno'].drop_duplicates().reset_index()['permno']
np.random.seed(42)
train_permnos = np.random.choice(permno_df, 4708, replace=False)
train_df = cleaned_df[cleaned_df['permno'].isin(train_permnos)]
test_df = cleaned_df[~cleaned_df['permno'].isin(train_permnos)]
train_df = train_df.drop(columns=['permno'])
test_df = test_df.drop(columns=['permno'])

# Allocate more memory
del ibes_df
del merged_df, permno_df, train_permnos
                          
# NaN imputation
to_remove = []
threshold = 0.5
for column in train_df.columns:
    
    missing_vals = train_df[column].isna().sum() + test_df[column].isna().sum()
    pct = missing_vals / (len(train_df[column]) + len(test_df[column]))
    
    if ((np.all(np.isnan(train_df[column])) or np.all(np.isnan(test_df[column]))) or pct >= threshold):
        to_remove.append(column)
    
    # Training set imputation
    train_mean = train_df[column].mean()
    train_df[column] = train_df[column].fillna(train_mean)

    # Testing set imputation
    test_mean = test_df[column].mean()
    test_df[column] = test_df[column].fillna(test_mean)

# Drop empty columns
train_df = train_df.drop(columns=to_remove)
test_df = test_df.drop(columns=to_remove)

# Split data into features and labels, then standardize according to training
y_train = train_df['AFE']
X_train = train_df.drop('AFE',axis=1)
y_test = test_df['AFE']
X_test = test_df.drop('AFE',axis=1)

standardizer = StandardScaler()
X_train = pd.DataFrame(standardizer.fit_transform(X_train))
X_test = pd.DataFrame(standardizer.transform(X_test))

In [ ]:
# Linear Regression model
lr_model = linear_model.LinearRegression()
lr_model.fit(X_train.to_numpy(),y_train.to_numpy())
y_hat_train = lr_model.predict(X_train.to_numpy())
train_score = r2_score(y_train.to_numpy(), y_hat_train)
print('Training R^2 Score (vanilla): ' + str(train_score))

# Evaluate linear model for all features on test data
y_hat_test = lr_model.predict(X_test.to_numpy())
test_score = r2_score(y_test.to_numpy(), y_hat_test)
print('Testing R^2 Score (vanilla): ' + str(test_score))

# Principal Component Analysis

In [ ]:
# Perform PCA (attempt 1)
pca = PCA(n_components=X_train.shape[1])
pca.fit_transform(X_train)
pc_vs_variance = np.cumsum(pca.explained_variance_ratio_)

# Plot cumulative variance as a function of PC's
plt.plot(pc_vs_variance)
plt.title('Principal Components vs Explained Variance')
plt.xlabel('No. of Components')
plt.ylabel('Cumulative Explained Variance')

# Select components which encompass 95% of explained variance
pca = PCA(n_components=np.argmax(pc_vs_variance>0.95))
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train linear model for PCs
lr_model_pca = linear_model.LinearRegression()
lr_model_pca.fit(X_train_pca,y_train.to_numpy())
y_hat_train_pca = lr_model_pca.predict(X_train_pca)
train_score_pca = r2_score(y_train.to_numpy(), y_hat_train_pca)
print('Training R^2 Score (PCA): ' + str(train_score_pca))

# Evaluate linear model for PCs on test data (transformed)
y_hat_test_pca = lr_model_pca.predict(X_test_pca)
test_score_pca = r2_score(y_test.to_numpy(), y_hat_test_pca)
print('Testing R^2 Score (PCA): ' + str(test_score_pca))

In [ ]:
# Perform PCA (attempt 2)
pca = PCA()
X_reduced_train = pca.fit_transform(X_train)
n = len(X_reduced_train)

mse = []
R2 = []

# Calculate MSE with only the intercept (no principal components in regression)
y_train_np = y_train.to_numpy()

# Calculate MSE using CV for the 19 principle components, adding one component at the time.
for i in np.arange(1, int(0.5*X_train.shape[1])):
    regr = linear_model.LinearRegression()
    regr.fit(X_reduced_train[:,:i], y_train_np)
    y_hat_trained = regr.predict(X_reduced_train[:,:i])
    
    mse_score = mean_squared_error(y_train_np, y_hat_trained)
    R2_score = r2_score(y_train_np, y_hat_trained)
    mse.append(mse_score)
    R2.append(R2_score)
    if (i % 10 == 0):
        print('No. of PCs: ' + str(i))

In [ ]:
# Plot comparison MSE as a function of number of PCs
mse = np.array(mse)
plt.plot(mse, '-v')
plt.xlabel('Number of principal components in regression')
plt.ylabel('MSE')

In [ ]:
# Plot comparison R2 as a function of number of PCs
R2 = np.array(R2)
plt.plot(R2, '-v')
plt.xlabel('Number of principal components in regression')
plt.ylabel('R2')

In [ ]:
# Use generated PCA transformation on test data
test_mse = []
test_R2 = []

y_test_np = np.array(y_test)

for i in range(1, int(0.5*X_train.shape[1])):
    X_reduced_test = pca.transform(X_test)[:,:i]

    # Train regression model on training data 
    regr = linear_model.LinearRegression()
    regr.fit(X_reduced_train[:,:i], y_train)

    # Prediction with test data
    pred = regr.predict(X_reduced_test)
    
    test_mse_score = mean_squared_error(y_test_np, pred)
    test_R2_score = r2_score(y_test_np, pred)
    test_mse.append(test_mse_score)
    test_R2.append(test_R2_score)
    if (i % 10 == 0):
        print('No. of PCs: ' + str(i))

In [ ]:
# Plot comparison test-MSE as a function of number of PCs
test_mse = np.array(test_mse)
plt.plot(test_mse, '-v')
plt.xlabel('Number of principal components in regression')
plt.ylabel('test MSE')

In [ ]:
# Plot comparison test-R2 as a function of number of PCs
test_R2 = np.array(test_R2)
plt.plot(test_R2, '-v')
plt.xlabel('Number of principal components in regression')
plt.ylabel('test R2')

# Classification of AFE

### Deciles

In [ ]:
# Split AFE into deciles
classification_df = cleaned_df
classification_df['AFE_decile'] = pd.qcut(classification_df['AFE'], 10, labels = False)
classification_df = classification_df.drop(columns=['AFE'])

# Split data into training and testing (grouped by company)
permno_df = classification_df['permno'].drop_duplicates().reset_index()['permno']
np.random.seed(42)
train_permnos = np.random.choice(permno_df, 4708, replace=False)
train_df = classification_df[classification_df['permno'].isin(train_permnos)]
test_df = classification_df[~classification_df['permno'].isin(train_permnos)]
train_df = train_df.drop(columns=['permno'])
test_df = test_df.drop(columns=['permno'])

# NaN imputation
to_remove = []
threshold = 0.5
for column in train_df.columns:
    
    missing_vals = train_df[column].isna().sum() + test_df[column].isna().sum()
    pct = missing_vals / (len(train_df[column]) + len(test_df[column]))
    
    if ((np.all(np.isnan(train_df[column])) or np.all(np.isnan(test_df[column]))) or pct >= threshold):
        to_remove.append(column)
    
    # Training set imputation
    train_mean = train_df[column].mean()
    train_df[column] = train_df[column].fillna(train_mean)

    # Testing set imputation
    test_mean = test_df[column].mean()
    test_df[column] = test_df[column].fillna(test_mean)

# Drop empty columns
train_df = train_df.drop(columns=to_remove)
test_df = test_df.drop(columns=to_remove)

# Split data into features and labels, then standardize according to training
y_train = train_df['AFE_decile']
X_train = train_df.drop('AFE_decile',axis=1)
y_test = test_df['AFE_decile']
X_test = test_df.drop('AFE_decile',axis=1)

standardizer = StandardScaler()
X_train = pd.DataFrame(standardizer.fit_transform(X_train))
X_test = pd.DataFrame(standardizer.transform(X_test))

In [ ]:
# Grid Search on depth and number of estimators (Random Forests)
param_grid_decile = {
    "n_estimators": [100, 200, 300, 500],
    "max_depth": [3, 5, 7, 9]
}
clf_decile = RandomForestClassifier()
grid_search_decile = GridSearchCV(estimator = clf_decile, param_grid = param_grid_decile, n_jobs = -1)
grid_search_decile.fit(X_train, y_train)
opt_params_decile_rf = grid_search_decile.best_params_
print(opt_params_decile_rf)

In [ ]:
# Random Forest implementation
# clf_decile_rf = RandomForestClassifier(n_estimators=opt_params_decile_rf['n_estimators'], max_depth=opt_params_decile_rf['max_depth'], random_state=42)
clf_decile_rf = RandomForestClassifier(n_estimators=500, max_depth=9, random_state=42)
clf_decile_rf.fit(X_train, y_train)
y_hat_train = clf_decile_rf.predict(X_train)
y_hat_test = clf_decile_rf.predict(X_test)
train_score = accuracy_score(y_train, y_hat_train)
test_score = accuracy_score(y_test, y_hat_test)
print("Tuned Random Forest (by decile) Train Score: " + str(train_score))
print("Tuned Random Forest (by decile) Test Score: " + str(test_score))

'''
# Feature importance
importances_decile = clf_decile_rf.feature_importances_
std_decile = np.std([tree.feature_importances_ for tree in clf_decile_rf.estimators_],
             axis=0)
indices_decile = np.argsort(importances_decile)[::-1]
plt.figure(figsize=(10, 8), dpi=80)
plt.title("Feature importances - Tuned RF (Deciles)")
first = 30
plt.bar(range(first), importances_decile[indices_decile[:first]],
        color="r", yerr=std_decile[indices_decile[:first]], align="center")
plt.xticks(range(first), indices_decile[:first-1])
plt.xlim([-1, first])
plt.show()
'''

In [ ]:
# Randomized Grid Search on depth and number of estimators (xgboost)

param_distributions_decile = {
    "n_estimators": [100, 200, 300, 400, 500],
    "max_depth": randint(low=1, high=10),
    "learning_rate": [0.1, 0.25, 0.5, 0.75],
    "subsample": [0.6, 0.8, 1.0]
}
xgb_decile = XGBClassifier(objective='multi:softmax', nthread=4, random_state=42)
rand_search_decile = RandomizedSearchCV(estimator = xgb_decile, param_distributions = param_distributions_decile, n_iter=100, n_jobs = -1, random_state=42, verbose=2)
rand_search_decile.fit(X_train, y_train)
opt_params_decile_xgb = rand_search_decile.best_params_
print(opt_params_decile_xgb)

In [ ]:
# XGBoost implementation
'''
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

param = {'max_depth': opt_params_decile_xgb['max_depth'], 'learning_rate': opt_params_decile_xgb['learning_rate'], 'objective': 'multi:softmax', }
param['nthread'] = 4
param['eval_metric'] = ['merror']
evallist = [(dtest, 'eval'), (dtrain, 'train')]

num_round = 100
bst = xgb.train(param, dtrain, num_round, evallist)
bst.save_model('0003.model')
y_hat_test = bst.predict(dtest)
y_hat_train = bst.predict(dtrain)
print()
print("XGBoost (by decile) Train Score: " + str(accuracy_score(y_train, y_hat_train)))
print("XGBoost (by decile) Test Score: " + str(accuracy_score(y_test,y_hat_test)))
'''

In [ ]:
# MultiLayer Perceptron (currently untuned)
'''
clf = MLPClassifier(hidden_layer_sizes=(256,256), activation='tanh', solver='adam', 
                    max_iter=100, random_state=42, verbose=True, early_stopping=True).fit(X_train, y_train)
y_hat_test = clf.predict(X_test)
y_hat_train = clf.predict(X_train)
train_score = clf.score(X_train, y_train)
test_score = clf.score(X_test, y_test)
print(train_score)
print(test_score)
plt.plot(clf.loss_curve_)
plt.title('Log-Loss over time for MLP (decile)')
plt.xlabel('Iterations')
plt.ylabel('Log-Loss')
'''

### Tertiles

In [ ]:
# Split AFE into tertiles
classification_df = cleaned_df
classification_df['AFE_tertile'] = pd.qcut(classification_df['AFE'], 3, labels = False)
classification_df = classification_df.drop(columns=['AFE'])

# Split data into training and testing (grouped by company)
permno_df = classification_df['permno'].drop_duplicates().reset_index()['permno']
np.random.seed(26)
train_permnos = np.random.choice(permno_df, 4708, replace=False)
train_df = classification_df[classification_df['permno'].isin(train_permnos)]
test_df = classification_df[~classification_df['permno'].isin(train_permnos)]
train_df = train_df.drop(columns=['permno'])
test_df = test_df.drop(columns=['permno'])
                          
# NaN imputation
to_remove = []
threshold = 0.5
for column in train_df.columns:
    
    missing_vals = train_df[column].isna().sum() + test_df[column].isna().sum()
    pct = missing_vals / (len(train_df[column]) + len(test_df[column]))
    
    if ((np.all(np.isnan(train_df[column])) or np.all(np.isnan(test_df[column]))) or pct >= threshold):
        to_remove.append(column)
    
    # Training set imputation
    train_mean = train_df[column].mean()
    train_df[column] = train_df[column].fillna(train_mean)

    # Testing set imputation
    test_mean = test_df[column].mean()
    test_df[column] = test_df[column].fillna(test_mean)

# Drop empty columns
train_df = train_df.drop(columns=to_remove)
test_df = test_df.drop(columns=to_remove)

# Split data into features and labels, then standardize according to training
y_train = train_df['AFE_tertile']
X_train = train_df.drop('AFE_tertile',axis=1)
y_test = test_df['AFE_tertile']
X_test = test_df.drop('AFE_tertile',axis=1)

standardizer = StandardScaler()
X_train = pd.DataFrame(standardizer.fit_transform(X_train))
X_test = pd.DataFrame(standardizer.transform(X_test))

In [ ]:
# Grid Search on depth and number of estimators (Random Forests)
param_grid_tertile = {
    "n_estimators": [100, 200, 300, 400, 500],
    "max_depth": [3, 5, 7, 9]
}
clf_tertile = RandomForestClassifier()
grid_search_tertile = GridSearchCV(estimator = clf_tertile, param_grid = param_grid_tertile, n_jobs = -1)
grid_search_tertile.fit(X_train, y_train)
opt_params_tertile_rf = grid_search_tertile.best_params_
print(opt_params_tertile_rf)

In [ ]:
# Random Forest implementation
# clf_tertile_rf = RandomForestClassifier(n_estimators=opt_params_tertile_rf['n_estimators'], max_depth=opt_params_tertile_rf['max_depth'], random_state=42)
clf_tertile_rf = RandomForestClassifier(n_estimators=400, max_depth=9, random_state=26)
clf_tertile_rf.fit(X_train, y_train)
y_hat_train = clf_tertile_rf.predict(X_train)
y_hat_test = clf_tertile_rf.predict(X_test)
train_score = accuracy_score(y_train, y_hat_train)
test_score = accuracy_score(y_test, y_hat_test)
print("Tuned Random Forest (by tertile) Train Score: " + str(train_score))
print("Tuned Random Forest (by tertile) Test Score: " + str(test_score))

# Confusion Matrix
disp = plot_confusion_matrix(clf_tertile_rf, X_test, y_test, cmap=plt.cm.Blues,normalize='true')
disp.ax_.set_title("Normalized Confusion Matrix")
print("Normalized Confusion Matrix")
print(disp.confusion_matrix)
plt.show()

In [ ]:
explainer = shap.TreeExplainer(clf_tertile_rf)
shap_values = explainer.shap_values(X_test)
# shap.summary_plot(shap_values, X_test, plot_type="bar")
shap.summary_plot(shap_values, X_test)

In [ ]:
print('Most Important Feature: ' + str(test_df.columns[221]))
print('2nd Most Important Feature: ' + str(test_df.columns[84]))
print('3rd Most Important Feature: ' + str(test_df.columns[61]))
print('4th Most Important Feature: ' + str(test_df.columns[64]))
print('5th Most Important Feature: ' + str(test_df.columns[62]))

In [ ]:
# Grid Search on depth and learning rate (xgboost)

param_distributions_tertile = {
    "max_depth": [6, 9],
    "learning_rate": [0.05, 0.1, 0.5],
}
xgb_tertile = XGBClassifier(n_estimators=400, verbosity=2, objective='multi:softmax', nthread=4, gamma=0.1, subsample=0.6, colsample_bytree=1.0, random_state=26)
grid_search_tertile = GridSearchCV(estimator = xgb_tertile, param_grid = param_distributions_tertile, n_jobs = -1, cv=3, verbose=3)
grid_search_tertile.fit(X_train, y_train)
opt_params_tertile_xgb = grid_search_tertile.best_params_
print(opt_params_tertile_xgb)

In [ ]:
# XGBoost implementation

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

param = {'max_depth': 9, 'eta': 0.05, 'objective': 'multi:softmax', 'num_class': 3, 'gamma': 0.2, 'subsample': 0.6, 'colsample_bytree': 1.0}
param['nthread'] = 4
param['verbosity'] = 3
param['eval_metric'] = ['merror']
evallist = [(dtest, 'eval'), (dtrain, 'train')]

num_round = 400
bst = xgb.train(param, dtrain, num_round, evallist)
bst.save_model('0004.model')
y_hat_test = bst.predict(dtest)
y_hat_train = bst.predict(dtrain)
print()
print('Tuned XGBoost (by tertile) training accuracy: ' + str(accuracy_score(y_train, y_hat_train)))
print('Tuned XGBoost (by tertile) testing accuracy: ' + str(accuracy_score(y_test,y_hat_test)))


In [ ]:
print()
print('Tuned XGBoost (by tertile) training accuracy: ' + str(accuracy_score(y_train, y_hat_train)))
print('Tuned XGBoost (by tertile) testing accuracy: ' + str(accuracy_score(y_test,y_hat_test)))
xgb.plot_importance(bst, max_num_features=15)

print()
print('Most Important Feature: ' + str(test_df.columns[1]))
print('2nd Most Important Feature: ' + str(test_df.columns[2]))
print('3rd Most Important Feature: ' + str(test_df.columns[228]))
print('4th Most Important Feature: ' + str(test_df.columns[12]))
print('5th Most Important Feature: ' + str(test_df.columns[11]))

In [ ]:
# Confusion Matrix
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
cm = confusion_matrix(y_test, y_hat_test, normalize='true')
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.title('Normalized Confusion Matrix')
#print("Normalized Confusion Matrix")
#print(disp.confusion_matrix)
#plt.show()